In [1]:
import sys
import os
import jieba
import config # 自定义配置文件
import numpy as np
import pandas as pd

# 不显示VisibleDeprecation警告
import warnings
warnings.filterwarnings("ignore", category=Warning)

### 导入数据

In [2]:
data = pd.read_csv(config.AF_Data_path + "AFAfter.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17651 entries, 0 to 17650
Data columns (total 16 columns):
Education     17651 non-null int64
Id            17651 non-null object
age           17651 non-null int64
gender        17651 non-null int64
query         17651 non-null object
SpaceNum      17651 non-null float64
SpaceRATIO    17651 non-null float64
LinkNum       17651 non-null float64
LinkRATIO     17651 non-null float64
TextSum       17651 non-null float64
TextMax       17651 non-null float64
TextMin       17651 non-null float64
TextMedian    17651 non-null float64
TextMean      17651 non-null float64
SearchNum     17651 non-null float64
HighWords     17651 non-null object
dtypes: float64(10), int64(3), object(3)
memory usage: 2.2+ MB


### 导入自定义词库

In [4]:
# 创建停用词列表
def stopwordslist():
    # 获取停用词文件列表
    for root, dirs, files in os.walk(config.stopword_path):
        pass
    # 停用词list
    stopwords = []
    for filename in files:
        for line in open(config.stopword_path + filename, 'r+', encoding='utf-8').readlines():
            stopword = line.strip()
            stopwords.append(stopword)
    return list(set(stopwords))

In [5]:
# 创建新增词汇
def add_Words():
    # 获取自定义词典文件名
    for root, dirs, files in os.walk(config.addword_path):
        filenames = [filename for filename in files if 'txt' in filename]
    addWords = []
    for filename in filenames:
        for line in open(config.addword_path + filename, 'r+').readlines():
            addword = line.strip()
            addWords.append(addword)
    return list(set(addWords))

In [6]:
addwordslist = add_Words() # 自定义词典
stopwordslist = stopwordslist() # 停用词词典

In [7]:
# jieba.add_word(word) # 新增词汇
jieba.load_userdict(addwordslist) # 添加用户自定义词典

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11147\AppData\Local\Temp\jieba.cache
Loading model cost 0.657 seconds.
Prefix dict has been built successfully.


### 生成词向量

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer # tfidf
# 模型、数据文件保存
from sklearn.externals import joblib
from sklearn.datasets import load_boston
import pickle

#### 停用词处理

In [8]:
def Deactivate_Words(words,stopwordslist):
    if len(words) <= 2:
        word_After_Stop = words
    elif len(words) > 2:
        word_After_Stop = list(set(words).difference(stopwordslist)) 
    return word_After_Stop

#### 分词处理

In [9]:
def get_tokens(wordtxt):
    tokens = [] # 用于存储分词结果
    for query in wordtxt.split('\t'): # 使用 \t 识别搜内容文本
        words = [word for word in jieba.cut(query)] # 分词
        words = Deactivate_Words(words,stopwordslist) # 停用词处理
#         print(words)
        # 分词结果进行拼接；强化词语语义
        for gram in [1,2]:
            for i in range(len(words) - gram + 1): # 根据分词结果列表进行拼接，最大拼接长度：2
                tokens += ["_*_".join(words[i:i+gram])] # 使用join拼接词语，并将结果添加至tokens
#                 print("_*_".join(words[i:i+gram]))
#         print("-"*20)
    return tokens

#### 生成搜索内容词向量

In [10]:
# 重写字符串标记化
class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line): # 可调用对象
        tokens = get_tokens(line)
        if np.random.rand() < 0.00001: # 随机输出一个切分样例
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%1000==0:
            print("已重写字符串记录数：" + str(self.n),end=' ')
        return tokens

In [12]:
# 原始文档集合转换为TF-IDF特征矩阵
TF_IDF = TfidfVectorizer(tokenizer=Tokenizer(), # 指定分割的方式,用于将字符串分割为一系列标记函数
                      min_df=3, # 
                      max_df=0.95, # 
                      sublinear_tf=True # 
                     ) # 实例化

In [13]:
X_sp = TF_IDF.fit_transform(data['query'].values.astype('U')) # 拟合模型并返回文本矩阵

已重写字符串记录数：1000 已重写字符串记录数：2000 已重写字符串记录数：3000 已重写字符串记录数：4000 已重写字符串记录数：5000 已重写字符串记录数：6000 九洲天空城结局	一世忘川	陈百强	重生不负	得非所愿by	揣着包子带球跑	得非所愿by忍冬	重生之总裁来爱我	imoo手机	星际第一海盗王	悄悄告诉他黑白剑妖	末世重生之累赘	极速前进第三季	买卖不成仁义在夏日	jackson周嘉诚	宋佳	蒲姓取名	我只喜欢你	君颜再归	光影边缘	我去上学啦2	腐书网2016	重生之太上皇嫁到	忠犬炼成手册	苹果7	买卖不成仁义在	香菇炖鸡	我不是死了吗白花花	杨幂	许韵珊	外星老公你走开	恶魔之名	戴向宇	吃什么补抗体	俏王爷强占山大王	色素之死性不改	王俊凯	章鱼不哭	蒲姓男孩最好的取名	重生之不可能的替身	论穿越到未来的风险性	周嘉诚	重生之有你相伴	重生之嫡子逆袭	禹成贤	从心开始	香菇	渣攻,不再见	ukiss kevin	赵文卓	周嘉诚天天向上	总有食材想吃掉我	极速前进	有匪美人来袭	一生相守	哪只总裁的替身	遇见你,要定你	放开我北鼻	重生之护夫狂魔	你的我的爱的礼物	炫浪网络社区	陈泰鸣	重生之我本君子	张国荣	你没事儿老瞅我干哈呀	相思不匪	重生之一盟情深	鸿星尔克官网	动漫电影	罗嘉良	七年之痒	安踏官方旗舰店	唐生	愿非所得	蝴蝶谷	2016综艺节目单	jackson	放开我北鼻jackson	重生之影帝家的古代萌妻	第一个故事谁爱他的妻子	仵作先生(穿越 悬疑推理 7)	冷总裁的贴身妖孽	无法泅渡	我不是死了吗	邓丽君	安全范围	重生之深藏不露	唐鹤德	梅艳芳	末世重生之丧尸大哥求放过	重生之陵渊求墨	白敬亭	腐书网	上司和我的秘密

['九洲', '天空', '结局', '城', '九洲_*_天空', '天空_*_结局', '结局_*_城', '一世', '忘川', '一世_*_忘川', '陈百强', '重生', '不负', '重生_*_不负', '非所愿', '揣着', '带球', '包子', '跑', '揣着_*_带球', '带球_*_包子', '包子_*_跑', '忍冬', '非所愿', '忍冬_*_非所愿', '总裁', '重生', '爱', '总裁_*_重生', '重生_*_爱', 'imoo', '手机', 'imo

#### 生成高频词词向量

In [15]:
# 原始文档集合转换为TF-IDF特征矩阵
TF_IDF_HW = TfidfVectorizer(min_df=3,max_df=0.95,sublinear_tf=True) # 实例化

In [16]:
HW_sp = TF_IDF_HW.fit_transform(data["HighWords"]) # 拟合模型并返回文本矩阵

### 持久化

#### 模型文件持久化

In [19]:
joblib.dump(TF_IDF,config.Model_path + 'TF_IDF.pkl')
joblib.dump(TF_IDF_HW,config.Model_path + 'TF_IDF_HW.pkl')

E:\Anaconda3_5.2.0\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
E:\Anaconda3_5.2.0\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
E:\Anaconda3_5.2.0\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
E:\Anaconda3_5.2.0\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
E:\Anaconda3_5.2.0\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


['./Model/TF_IDF_HW.pkl']

In [20]:
#载入模型
TF_IDF_Model = joblib.load(config.Model_path + 'TF_IDF.pkl')
TF_IDF_HW_Model = joblib.load(config.Model_path + 'TF_IDF_HW.pkl')

#### 数据文件持久化

In [21]:
# 保存数据文件
pkl_file = open(config.WV_Data_path + 'TFIDF_sp.feat','wb')
pickle.dump(X_sp,pkl_file)
pkl_file.close()

In [24]:
# 保存数据文件
pkl_file = open(config.WV_Data_path + 'TFIDF_HW_sp.feat','wb')
pickle.dump(HW_sp,pkl_file)
pkl_file.close()

In [22]:
# 加载文件
pkl_file = open(config.WV_Data_path + 'TFIDF_sp.feat','rb')
T_X_sp= pickle.load(pkl_file)
pkl_file.close()

In [25]:
# 加载文件
pkl_file = open(config.WV_Data_path + 'HW_sp.feat','rb')
T_HW_sp= pickle.load(pkl_file)
pkl_file.close()

#### 向量化数据文件持久化

In [10]:
TFIDF_sp_toarray = TFIDF_sp.toarray()

In [29]:
TFIDF_HW_sp_toarray = TFIDF_HW_sp.toarray()

In [13]:
with h5py.File(config.WV_Data_path + "TFIDF_sp_toarray.hdf5", 'w') as f:
    f.create_dataset("TFIDF_sp_toarray", # TFIDF_sp_toarray:是HDF5文件中c数据储存的名字
                     data=TFIDF_sp_toarray,  # 高维数组
                     compression="gzip", # compression="gzip"：是对数据进行压缩，压缩的方式一个gzip
                     compression_opts=5 # compression_opts=5：是压缩等级，压缩等级越高，压缩的越好，但是压缩消耗的CPU时间会增加
                    )

In [10]:
# 保存数据文件
pkl_file = open(config.WV_Data_path + 'TFIDF_sp_toarray.feat','wb')
pickle.dump(TFIDF_sp_toarray,pkl_file)
pkl_file.close()

In [ ]:
# 加载文件
with h5py.File(config.WV_Data_path + "TFIDF_sp_toarray.hdf5", 'r') as f:  # 读取的时候是‘r’
    print(f.keys())
    TFIDF_sp_toarray = f.get("TFIDF_sp_toarray")[:]

In [ ]:
# 加载文件
pkl_file = open(config.WV_Data_path + 'TFIDF_HW_sp_toarray.feat','rb')
TFIDF_HW_sp_toarray= pickle.load(pkl_file)
pkl_file.close()